In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import cv2
import numpy as np
import imutils
import random
import pickle
import os
from math import floor

In [ ]:
data_dir = Path.cwd() / "EnsembleB"

# A list of string with all the categories/labels in your database, i.e., each class subfolder name
LABELS = ["Cercles2", "Cercles5", "Hexagones2","Hexagones5", "Triangles2","Triangles5" ,"Diamants2", "Diamants5"]

In [ ]:
# Image resize
WIDTH = 160
HEIGHT = 70

# Dataset: this list will contain all the images
data_set = []
dataset_entrainement = []
dataset_validation = []
dataset_tester = []

In [ ]:
def detect_shape(contour):
    # Initialize the shape name and approximate the contour
    shape = "unidentified"
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * peri, True)
    
    # Determine the shape based on the number of vertices
    if len(approx) == 3:
        shape = "triangle"
    elif len(approx) == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        ar = w / float(h)
        shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
    elif len(approx) == 5:
        shape = "pentagon"
    elif len(approx) == 6:
        shape = "hexagon"
    else:
        # For shapes with more than 6 vertices, we can assume it's a circle
        shape = "circle"

    # Return the name of the shape
    return shape


In [ ]:
for first_level in data_dir.glob('*'):
    if first_level.is_dir():
        for sec_level in first_level.glob('*'):
            label = sec_level.name
            if label in LABELS:
                class_num = LABELS.index(label)
                print(f"{label} {class_num}")
                for image_path in sec_level.glob('*'):
                    # Read the image with OpenCV
                    image = cv2.imread(str(image_path))
                    
                    # Check if the image is loaded correctly
                    if image is not None:
                        # Resize the image
                        resized_image = cv2.resize(image, (WIDTH, HEIGHT))
                        
                        # Convert the resized image to grayscale
                        gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                        
                        # Apply Gaussian blur
                        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
                        
                        

                        # # Detect edges
                        # edges = cv2.Canny(blurred_image, 50, 150)
                        
                        # # Find contours
                        # contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        
                        # # Loop over the contours and detect shapes
                        # for contour in contours:
                        #     shape = detect_shape(contour)
                            
                        #     # Optionally draw the detected shape on the image
                        #     cv2.drawContours(resized_image, [contour], -1, (0, 255, 0), 2)
                            
                        #     # Calculate the centroid to place the shape name
                        #     M = cv2.moments(contour)
                        #     if M["m00"] != 0:
                        #         cX = int(M["m10"] / M["m00"])
                        #         cY = int(M["m01"] / M["m00"])
                        #         cv2.putText(resized_image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                                
                        # cv2.imshow("Detected Shapes", resized_image)
                        # cv2.waitKey(0)
                        # cv2.destroyAllWindows()
                        #                         # Add information to the dataset
                        data_set.append([resized_image, class_num, label])
                        
                        # Break after processing the first image in the folder
                        
                
        

# Optional: Display the last processed image


In [ ]:
def sauvegarde_donnees(dataset, tlisteLabelspeDeDonnees):
    # Permet de melanger aleatoirement toutes les images
    random.shuffle(dataset)

    listeImages = [] # listes des images
    listeLabels = [] # listes des labels

    # recupperer les images et les labels
    for features, class_num, label in dataset:
        listeImages.append(features)
        listeLabels.append(label)

    # Converts each image matrilisteImages to an image vector
    listeImages = np.arralisteLabels(listeImages).reshape(-1, WIDTH, HEIGHT, 1)

    nomFichierImages = ''
    nomFicherLabels = ''
    
    if (tlisteLabelspeDeDonnees == 'V'):
        nomFichierImages = 'listeImages_validation.pickle'
        nomFicherLabels = 'listeLabels_validation.pickle' 
    elif (tlisteLabelspeDeDonnees == 'E'):
        nomFichierImages = 'listeImages_entrainement.pickle'
        nomFicherLabels = 'listeLabels_entrainement.pickle' 
    elif (tlisteLabelspeDeDonnees == 'Test'):
        nomFichierImages = 'listeImages_test.pickle'
        nomFicherLabels = 'listeLabels_test.pickle'
    
    # Creating the files containing all the information about listeLabelsour model and saving them to the disk
    pickle_out = open(f"donnees_sauvegarder/{nomFichierImages}", "wb")
    pickle.dump(listeImages, pickle_out)
    pickle_out.close()

    pickle_out = open(f"donnees_sauvegarder/{nomFicherLabels}", "wb")
    pickle.dump(listeLabels, pickle_out)
    pickle_out.close()

In [ ]:
def augmententation(dataset, image, class_num, label):
    #On utilise les differentes methodes fournis dans les tutoriels pour faire laugmentation
    imageFlippe = cv2.flip(image, 1)
    # Getting image size
    w, h  = image.shape[:2]
    # Image center
    center = (w / 2, h / 2)
    # Scaling factor
    scale = 1

    for angle in range(0, 360, 15):
        M = cv2.getRotationMatrix2D(center, angle, scale)
        imageAugmente = cv2.warpAffine(image, M, (w, h))
        imageFlippeAugmente = cv2.warpAffine(imageFlippe, M, (w, h))

        dataset.append([imageAugmente, class_num, label])
        dataset.append([imageFlippeAugmente, class_num, label])

In [ ]:
def divide_images(images, class_num, label, pourcentageEntrainement):
    random.shuffle(images)
    counter = 0

    pourcentageRestant = pourcentageEntrainement + ((1-pourcentageEntrainement)/2)
    # pourcentageValidation = (1-pourcentageEntrainement)/2
    # pourcentageTest = (1-pourcentageEntrainement)/2

    for image in images:
        # 60% Training
        if (counter in range(0, floor(0.6*len(images)))):
            augmente(training_data_set, image, class_num, label)
        # 20% Validation
        elif (counter in range(floor(0.6*len(images)), floor(0.8*len(images)))):
            augmente(validation_data_set, image, class_num, label)
        # 20% Test
        elif(counter in range(floor(0.8*len(images)), len(images))):
            augmente(test_data_set, image, class_num, label)
        counter+=1
